In [76]:
import numpy as np
import pandas as pd

In [77]:
directory="/home/benjamin/PycharmProjects/UCL Hackathon/Hackathon/"
chi_files=["chi_vals_2020-12-01_2030-11-30.txt","chi_vals_2030-12-01_2040-11-30.txt","chi_vals_2040-12-01_2050-11-30.txt"]
mask_files=["land_mask_ind_2020-12-01_2030-11-30.txt","land_mask_ind_2030-12-01_2040-11-30.txt","land_mask_ind_2040-12-01_2050-11-30.txt"]
lat_files=["lat_coords_2020-12-01_2030-11-30.txt","lat_coords_2030-12-01_2040-11-30.txt","lat_coords_2040-12-01_2050-11-30.txt"]
lon_files=["lon_coords_2020-12-01_2030-11-30.txt","lon_coords_2030-12-01_2040-11-30.txt","lon_coords_2040-12-01_2050-11-30.txt"]


In [78]:
def string_toindex(string):
    first_space_index=string.find(' ')
    second_space_index=string.find(' ',first_space_index+1)
    return int(string[0:first_space_index])-1, (int(string[first_space_index:second_space_index]),int(string[second_space_index:]))

In [79]:
def load_chitwod(decade):
    chi_values=np.loadtxt(directory+chi_files[decade])
    df = pd.read_csv(directory+mask_files[decade], delimiter = "\t")
    landmass_index=df.to_numpy()
    chi_twod=np.full((128,128,2064),-1,dtype='f8')
    twod_indices=[]
    for x in landmass_index:
        x_oned_index, x_twod_index= string_toindex(x[0])
        twod_indices.append(x_twod_index)
        for k in range(len(chi_values)):
        #    for y in landmass_index:
        #        y_oned_index, y_twod_index= string_toindex(y[0])
            chi_twod[x_twod_index+(k,)]=chi_values[x_oned_index,k]
    return chi_twod, twod_indices

In [80]:
import numpy as np
from scipy.ndimage.measurements import label

In [81]:
def size_connected_component(array, threshhold, centre_twod, diagonal_neighbourhood=False):
    if diagonal_neighbourhood==True:
        structure = np.ones((3, 3), dtype=np.int) 
    else:
        structure = np.array([[0,1,0],[1,1,1],[0,1,0]])
    boolean_array=(array>threshhold).astype(int)
    labeled, ncomponents = label(boolean_array, structure)
    label_centre=labeled[centre_twod]
    return np.count_nonzero(labeled==label_centre)   

In [82]:
def size_cc_chivalues(chi_twod,twod_indices,threshhold, centre, diagonal_neighbourhood=False):
    array=chi_twod[:,:,centre]
    centre_twod=twod_indices[centre]
    return size_connected_component(array, threshhold, centre_twod, diagonal_neighbourhood)

In [83]:
def size_cc_global(decade,thresshold, diagonal_neighbourhood=False):
    chi_twod, twod_indices=load_chitwod(decade)
    N=len(twod_indices)
    all_cc_values=[size_cc_chivalues(chi_twod,twod_indices,thresshold, centre, diagonal_neighbourhood) for centre in range(N)]
    return np.mean(all_cc_values)

In [110]:
first_decade=size_cc_global(decade=0,thresshold=0.9)*144
second_decade=size_cc_global(decade=1,thresshold=0.9)*144
third_decade=size_cc_global(decade=2,thresshold=0.9)*144

In [88]:
def conv_oned_tolatlong(decade,k):
    twod_indices=[]
    df = pd.read_csv(directory+mask_files[decade], delimiter = "\t")
    landmass_index=df.to_numpy()
    for x in landmass_index:
        x_oned_index, x_twod_index= string_toindex(x[0])
        twod_indices.append(x_twod_index)
    x,y =twod_indices[k]
    lats=np.loadtxt(directory+lat_files[decade])
    longs=np.loadtxt(directory+lon_files[decade])
    lat=lats[x]
    long=longs[y]
    return lat, long

In [111]:
print(first_decade)
print(second_decade)
print(third_decade)

2452.813953488372
2153.093023255814
1796.7906976744187
